# Receiving C++ Data and Transforming to ArcGIS Online #

 First, import ArcGIS 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from arcgis.widgets import MapView
from arcgis.features import FeatureLayer
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureCollection
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geocoding import geocode
from arcgis.features.analysis import join_features
from scipy import stats

In [107]:
gis = GIS(
  url="https://www.arcgis.com",
  username="jboothhowe",
  password="Jujubeads777!")

In [111]:
data_properties = {
  "title": "Environmental Health",
  "description": "Environmental Health data imported from C++ function",
  "tags": "Boston MA"
}
csv_filepath = "/Users/jboothhowe/Documents/CS2300/Final-Project/cmake-build-debug/DataToMap.csv"
data = pd.read_csv(csv_filepath)
csv_item = gis.content.add(data_properties, csv_filepath)
boston_health = csv_item.publish()

In [134]:
# Extract variables GeoID, UrbanHeat_Variable, SVI_Variable from DataFrame
geoID = data['GeoID'].tolist()  # Convert GeoID column to list
urban_heat = data['UrbanHeat_Variable'].tolist()  # Convert UrbanHeat_Variable column to list
svi_variable = data['SVI_Variable'].tolist()  # Convert SVI_Variable column to list

In [116]:
## Now importing the shapefile for the Boston 2010 census
boston_census = {
    'title': 'Boston 2010 Census Tracts',
    'tags':'boston, 2010, census',
    'type': 'Shapefile'
}

In [117]:
data_file_location = "/Users/jboothhowe/Documents/CS2300/Final-Project/Census_2010_Tracts.zip" # relative path to notebook
shp_item = gis.content.add(boston_census, data=data_file_location)
shp_item

<Item title:"Boston 2010 Census Tracts" type:Shapefile owner:jboothhowe>

In [118]:
boston_census_feature_layer = shp_item.publish()
boston_census_feature_layer

<Item title:"Boston 2010 Census Tracts" type:Feature Layer Collection owner:jboothhowe>

In [119]:
m = gis.map("Boston, MA")
m.add_layer(boston_census_feature_layer)
m

MapView(layout=Layout(height='400px', width='100%'))

In [130]:
# Convert 'GEOID10' field to a long integer
boston_census_layer = boston_census_feature_layer.layers[0]  # Assuming boston_census_feature_layer is a feature layer
boston_census_layer.filter = "1=1"  # Retrieve all features
features = boston_census_layer.query(return_ids_only=False).features
for feature in features:
    try:
        feature.attributes['GEOID10'] = int(feature.attributes['GEOID10'])
    except ValueError:
        # Handle non-integer values if necessary
        pass

In [133]:
targetLayer = boston_census_layer

joinLayer = boston_health

# Perform join operation
results = join_features(
    target_layer=targetLayer,
    join_layer=joinLayer,
    join_operation="joinOneToOne",
    attribute_relationship=[
        {
            "targetField": "GEOID10",
            "operator": "equal",
            "joinField": "GeoID",
        }
    ],
    output_name="Join features"
)
join_features(target_layer, join_layer, spatial_relationship, 
              spatial_relationship_distance, spatial_relationship_distance_units,
              attribute_relationship, join_operation, summary_fields, output_name, context, gis, estimate, future, join_type, records_to_match)

AttributeError: 'NoneType' object has no attribute '_tbx'